In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import cftime
import calendar
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import matplotlib.ticker as mticker
from matplotlib.colors import TwoSlopeNorm
import imageio
import os
import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [18]:
window_before = 5
window_after = 10

In [19]:
t2m_wf1 = xr.open_dataset('/nird/datalake/NS1004K/elihho/tes0004_echam6_BOT_mm_0_1850_var167.nc')
t2m_wof1 = xr.open_dataset('/nird/datalake/NS1004K/elihho/slo0059_echam6_BOT_mm_0_1850_var167.nc')
tas_orb_GHG = xr.open_dataset('slo0043_echam6_BOT_mm_1001-8850_167_NH.nc')
tas_all_forcing = xr.open_dataset('slo0042+slo0046+slo0050_echam6_BOT_mm_1001_8850_167_NH.nc')

In [20]:
# For t2m_wf
n_months_wf = t2m_wf1.sizes['time']
years_wf = np.floor_divide(np.arange(n_months_wf), 12)
months_wf = (np.remainder(np.arange(n_months_wf), 12) + 1)
cftime_time_wf = [cftime.DatetimeNoLeap(int(y), int(m), 1) for y, m in zip(years_wf, months_wf)]
t2m_wf = t2m_wf1.assign_coords(time=("time", cftime_time_wf))

# For t2m_wof
n_months_wof = t2m_wof1.sizes['time']
years_wof = np.floor_divide(np.arange(n_months_wof), 12)
months_wof = (np.remainder(np.arange(n_months_wof), 12) + 1)
cftime_time_wof = [cftime.DatetimeNoLeap(int(y), int(m), 1) for y, m in zip(years_wof, months_wof)]
t2m_wof = t2m_wof1.assign_coords(time=("time", cftime_time_wof))

# For orb_GHG
n_months_wf = tas_orb_GHG.sizes['time']
start_year_wf = 1850 - (n_months_wf // 12)  # reverse from last year
years_wf = np.floor_divide(np.arange(n_months_wf), 12) + start_year_wf
months_wf = (np.remainder(np.arange(n_months_wf), 12) + 1)
cftime_time_wf = [cftime.DatetimeNoLeap(int(y), int(m), 1) for y, m in zip(years_wf, months_wf)]
tas_orb_GHG = tas_orb_GHG.assign_coords(time=("time", cftime_time_wf))

# For all_forcing 
n_months_wof = tas_all_forcing.sizes['time']
start_year_wof = 1850 - (n_months_wof // 12)
years_wof = np.floor_divide(np.arange(n_months_wof), 12) + start_year_wof
months_wof = (np.remainder(np.arange(n_months_wof), 12) + 1)
cftime_time_wof = [cftime.DatetimeNoLeap(int(y), int(m), 1) for y, m in zip(years_wof, months_wof)]
tas_all_forcing = tas_all_forcing.assign_coords(time=("time", cftime_time_wof))

In [21]:
start_year = 1251
end_year = 1850

t2m_wf_lia = t2m_wf.sel(time=slice(f'{start_year}-01-01', f'{end_year}-12-31'))
t2m_wof_lia = t2m_wof.sel(time=slice(f'{start_year}-01-01', f'{end_year}-12-31'))
tas_orb_GHG_lia = tas_orb_GHG.sel(time=slice(f'{start_year}-01-01', f'{end_year}-12-31'))
tas_all_forcing_lia = tas_all_forcing.sel(time=slice(f'{start_year}-01-01', f'{end_year}-12-31'))

In [22]:
scand_lat = slice(89, 50)
scand_lon = slice(-10, 50)

scand_lia_wf = t2m_wf_lia['var167'].sel(lat=scand_lat, lon = scand_lon).mean(dim=['lat', 'lon'])
scand_lia_wof = t2m_wof_lia['var167'].sel(lat=scand_lat, lon = scand_lon).mean(dim=['lat', 'lon'])
scand_orb_GHG_lia = tas_orb_GHG_lia['var167'].sel(lat=scand_lat, lon = scand_lon).mean(dim=['lat', 'lon'])
scand_all_forcing_lia = tas_all_forcing_lia['var167'].sel(lat=scand_lat, lon = scand_lon).mean(dim=['lat', 'lon'])

rolling_year=10
rolling_months = rolling_year*12
# Smooth with rolling mean
scand_smoothed_wf = scand_lia_wf.rolling(time=rolling_months, center=True).mean()
scand_smoothed_wof = scand_lia_wof.rolling(time=rolling_months, center=True).mean()
scand_orb_GHG_smooth = scand_orb_GHG_lia.rolling(time=rolling_months, center=True).mean()
scand_all_forcing_smooth = scand_all_forcing_lia.rolling(time=rolling_months, center=True).mean()

scand_lia_wf

<xarray.DataArray 'var167' (time: 6948)> Size: 28kB
array([260.85742, 263.4397 , 259.37662, ..., 271.23566, 266.32053,
       261.1772 ], dtype=float32)
Coordinates:
  * time     (time) object 56kB 1251-01-01 00:00:00 ... 1829-12-01 00:00:00

In [23]:
years = scand_lia_wf['time'].dt.year.values
eruption_years = [1257, 1452, 1600, 1815]  

In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def superposed_epoch_analysis_simple(years, data, eruption_years, window_before=5, window_after=10):
    all_epochs = []

    for eruption in eruption_years:
        start = eruption - window_before
        end = eruption + window_after

        # Check if the window fits within the data range
        if start < years[0] or end > years[-1]:
            continue

        # Extract data for the window
        mask = (years >= start) & (years <= end)
        window_years = years[mask]
        window_data = data[mask]

        if len(window_data) != window_before + window_after + 1:
            continue  # Skip incomplete windows

        # Convert to anomaly relative to pre-eruption mean
        baseline = window_data[:window_before].mean()
        anomaly = window_data - baseline

        all_epochs.append(anomaly)

    if not all_epochs:
        raise ValueError("No valid eruption windows found.")

    all_epochs = np.array(all_epochs)
    mean_anomaly = all_epochs.mean(axis=0)
    std_anomaly = all_epochs.std(axis=0)

    rel_years = np.arange(-window_before, window_after + 1)

    return rel_years, mean_anomaly, std_anomaly, all_epochs


In [27]:
# Prepare year and data arrays
years = scand_lia_wf['time'].dt.year.values
wf_values = scand_lia_wf.values
wof_values = scand_lia_wof.values
orb_GHG_values = scand_orb_GHG_lia.values
all_forcing_values = scand_all_forcing_lia.values

valid_mask = ~np.isnan(wof_values)
years_clean = years[valid_mask]
wf_values_clean = wf_values[valid_mask]


IndexError: boolean index did not match indexed array along dimension 0; dimension is 6948 but corresponding boolean dimension is 7188

In [28]:
rel_years, mean_anomaly, std_anomaly, all_epochs = superposed_epoch_analysis_simple(
    years_clean, wf_values_clean, eruption_years, window_before=5, window_after=10
)


ValueError: No valid eruption windows found.